In [1]:
import pandas as pd

In [60]:
data_original = pd.read_csv("./pretest_data.csv")
data_original['title']

0                        만 19세 '연봉 1억' 친구와 레스토랑 브이로그
1                               BBQ 신메뉴 '체고치' 순살 먹어봄
2                                     구찌 라이톤이랑 시계 후기
3                                엽기 로제떡볶이랑 허니 콤보 혼내줌
4                                 어림도 없지 바로 마라탕 먹어버림
                             ...                    
101159                     '현실남매' 먹방하러 갔다가 진짜 싸웠습니다.
101160    [직업체험] DJ예나 일일 매니저 (월디페/World DJ Festival)
101161                           저의 슈퍼모델 친구들을 소개합니다.
101162             '현실남매' 우리만 이런건가요? 남들이 이해 못하는 현실남매
101163          유튜브 시작하기 "일반인이 유튜브를 시작하면 가장 많이하는 고민"
Name: title, Length: 101164, dtype: object

In [43]:
punct1 = "/-'.,#$\'()*+-/:;<=>@[\\]^_`{|}~%" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct2 = "!?|│" #not remove
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "",
                 "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2",
                 "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"',
                 '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta',
                 '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-',
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    for p in punct1:
        text = text.replace(p, f' ')
    for p in punct2:
        text = text.replace(p, f' {p} ')
        
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text.strip()

In [44]:
data_clean_punc = []
for title in data_original['title']:
    data_clean_punc.append(clean_punc(title, punct, punct_mapping))

In [63]:
import re
import emoji

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '', str(texts[i])) #remove punctuation
        #review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        #review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        review = emoji.demojize(review)
        corpus.append(review)
        
    return corpus

In [ ]:
data_clean_text = clean_text(data_clean_punc)
data_clean_text